In [4]:
!pip install segmentation_models_pytorch
!pip install -q -U albumentations
!pip install timm
!pip install adamp
!pip install neptune-client
!pip install fastai --upgrade

     |████████████████████████████████| 71kB 6.5MB/s 
     |████████████████████████████████| 61kB 7.7MB/s 
     |████████████████████████████████| 245kB 19.8MB/s 
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-cp37-none-any.whl size=60963 sha256=36b72d9a292e8f969caeefee1e11416ee2d688cf7fa2c0fb81b978bf9b2af9bf
  Stored in directory: /root/.cache/pip/wheels/69/df/63/62583c096289713f22db605aa2334de5b591d59861a02c2ecd
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.6.3-cp37-none-any.whl size=12420 sha256=1fbf19a89affa14de03899a358ebc99536fe64d8e6a398a8764d533a28c476fc
  Stored in directory: /root/.cache/pip/wheels/42/1e/a9/2a578ba9ad04e776e80bf0f70d8a7f4c29ec0718b92d8f6ccd
Successfully built pretrainedmodels efficientnet-pytorch
     |████████████████████████████████| 81kB 7.6MB/s 
     |████████████████████████████████| 37.6MB 126kB/s 
     |████████████████████████████████| 952kB 59.1MB/s 
  Created wheel for adamp: filename=adamp-0.3.0-c

In [4]:
!nvidia-smi

Wed May  5 04:51:55 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    24W / 300W |      2MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
import segmentation_models_pytorch as smp
from tqdm import tqdm
import gc
from adamp import AdamP
from typing import Optional

import math
from torch.optim.optimizer import Optimizer, required

from fastai.vision.all import *

from sklearn.model_selection import GroupKFold, KFold
import torch
from torch import nn
import torchvision
import cv2
import os
import numpy as np
import pandas as pd

from torchvision import transforms
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.cuda.amp import autocast, GradScaler
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau, CosineAnnealingWarmRestarts, _LRScheduler
from scipy.ndimage.interpolation import zoom
import albumentations as A
from torch.nn import functional as F
from albumentations.pytorch import ToTensorV2

from pycocotools.coco import COCO

import matplotlib.pyplot as plt
import sys
import time
import random
import timm

import neptune
# import encoding

In [2]:
# sys.path.append('/content/drive/MyDrive/trash_segmentation/data/segmentation_models.pytorch-master')

# import segmentation_models_pytorch as smp

In [3]:
# run = neptune.init('vvvic313/trash-segmentation', api_token='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJlODg2NjVlNC01YjIxLTQ3ZGItYWVkYS05MGFiYWNjMGI2YjUifQ==')

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [5]:
CFG = {
    "img_size": 512,
    "num_workers": 4,
    "scheduler": "Warmup",
    "epochs": 25,
    "criterion": "SoftCELoss",
    "decoder": "UneXt",
    "encoder": "resnext50",
    "pretrained": "swsl",
    "lr": 1e-4,
    "batch_size": 8,
    "weight_decay": 1e-6,
    "gradient_accumulation_steps": 4,
    "seed": 42,
    "optimizer": "AdamW",
    "mean": (0.485, 0.456, 0.406),
    "std": (0.229, 0.224, 0.225),
    "mix_prob": 0,
    "pseudo_label": True
}

In [6]:
seed_everything(CFG['seed'])

In [7]:
def get_train_augmentations():
    return A.Compose([
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.ShiftScaleRotate(p=0.5),
        A.Cutout(),
        A.Normalize(mean=CFG['mean'], std=CFG['std'], max_pixel_value=255.0, p=1.0),
        ToTensorV2()
    ], p=1.0)

In [8]:
# neptune.create_experiment(name="day_0502", params=CFG)
# neptune.append_tag("effb3", "deeplabv3+",  "heavy_aug", "fold2")

In [9]:
def get_validation_augmentations():
    return A.Compose([
        A.Normalize(mean=CFG['mean'], std=CFG['std'], max_pixel_value=255.0, p=1.0),
        ToTensorV2()
    ],p=1.0)

In [10]:
class TrashDataset(Dataset):
    def __init__(self, df, root="/content/drive/MyDrive/trash_segmentation/data/", mode="train", transform=None):
        self.df = df.reset_index(drop=True).copy()
        self.mode = mode
        self.transform = transform
        self.root = root
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        image_path = self.root + self.df.iloc[idx]['filepath']
        imgs = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
        
        
        if self.mode=="train" or self.mode=="val":
            mask_path = self.root + self.df.iloc[idx]['masks']
            masks = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE).astype(np.float32)
            transformed = self.transform(image=imgs, mask=masks)
            imgs = transformed["image"]
            masks = transformed["mask"]

            return imgs, masks
        
        elif self.mode == "test":
            transformed = self.transform(image=imgs)
            imgs = transformed["image"]
            
            return imgs, 1

In [11]:
df = pd.read_csv("/content/drive/MyDrive/trash_segmentation/data/train.csv")

if CFG['pseudo_label']:
    additional_df = pd.read_csv("/content/drive/MyDrive/trash_segmentation/data/test.csv")
    df = pd.concat([df, additional_df], ignore_index=True)

In [12]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [13]:
def prepare_dataloader(df, fold):
    train_ids = df[~df.Folds.isin(fold)]
    val_ids = df[df.Folds.isin(fold)]

    train_ds = TrashDataset(train_ids, mode="train", transform=get_train_augmentations())
    val_ds = TrashDataset(val_ids, mode="val", transform=get_validation_augmentations())
    
    train_loader = DataLoader(train_ds, 
                              batch_size=CFG["batch_size"], 
                              shuffle=True, 
                              num_workers=CFG["num_workers"],
                              collate_fn=collate_fn)
    val_loader = DataLoader(val_ds,
                            batch_size=CFG["batch_size"],
                            shuffle=False,
                            num_workers=CFG["num_workers"],
                            collate_fn=collate_fn)
    
    return train_loader, val_loader

In [14]:
class FPN(nn.Module):
    def __init__(self, input_channels:list, output_channels:list):
        super().__init__()
        self.convs = nn.ModuleList(
            [nn.Sequential(nn.Conv2d(in_ch, out_ch*2, kernel_size=3, padding=1),
             nn.ReLU(inplace=True), nn.BatchNorm2d(out_ch*2),
             nn.Conv2d(out_ch*2, out_ch, kernel_size=3, padding=1))
            for in_ch, out_ch in zip(input_channels, output_channels)])
        
    def forward(self, xs:list, last_layer):
        hcs = [F.interpolate(c(x),scale_factor=2**(len(self.convs)-i),mode='bilinear') 
               for i,(c,x) in enumerate(zip(self.convs, xs))]
        hcs.append(last_layer)
        return torch.cat(hcs, dim=1)

class UnetBlock(Module):
    def __init__(self, up_in_c:int, x_in_c:int, nf:int=None, blur:bool=False,
                 self_attention:bool=False, **kwargs):
        super().__init__()
        self.shuf = PixelShuffle_ICNR(up_in_c, up_in_c//2, blur=blur, **kwargs)
        self.bn = nn.BatchNorm2d(x_in_c)
        ni = up_in_c//2 + x_in_c
        nf = nf if nf is not None else max(up_in_c//2,32)
        self.conv1 = ConvLayer(ni, nf, norm_type=None, **kwargs)
        self.conv2 = ConvLayer(nf, nf, norm_type=None,
            xtra=SelfAttention(nf) if self_attention else None, **kwargs)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, up_in:Tensor, left_in:Tensor) -> Tensor:
        s = left_in
        up_out = self.shuf(up_in)
        cat_x = self.relu(torch.cat([up_out, self.bn(s)], dim=1))
        return self.conv2(self.conv1(cat_x))
        
class _ASPPModule(nn.Module):
    def __init__(self, inplanes, planes, kernel_size, padding, dilation, groups=1):
        super().__init__()
        self.atrous_conv = nn.Conv2d(inplanes, planes, kernel_size=kernel_size,
                stride=1, padding=padding, dilation=dilation, bias=False, groups=groups)
        self.bn = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU()

        self._init_weight()

    def forward(self, x):
        x = self.atrous_conv(x)
        x = self.bn(x)

        return self.relu(x)

    def _init_weight(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                torch.nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

class ASPP(nn.Module):
    def __init__(self, inplanes=512, mid_c=256, dilations=[6, 12, 18, 24], out_c=None):
        super().__init__()
        self.aspps = [_ASPPModule(inplanes, mid_c, 1, padding=0, dilation=1)] + \
            [_ASPPModule(inplanes, mid_c, 3, padding=d, dilation=d,groups=4) for d in dilations]
        self.aspps = nn.ModuleList(self.aspps)
        self.global_pool = nn.Sequential(nn.AdaptiveMaxPool2d((1, 1)),
                        nn.Conv2d(inplanes, mid_c, 1, stride=1, bias=False),
                        nn.BatchNorm2d(mid_c), nn.ReLU())
        out_c = out_c if out_c is not None else mid_c
        self.out_conv = nn.Sequential(nn.Conv2d(mid_c*(2+len(dilations)), out_c, 1, bias=False),
                                    nn.BatchNorm2d(out_c), nn.ReLU(inplace=True))
        self.conv1 = nn.Conv2d(mid_c*(2+len(dilations)), out_c, 1, bias=False)
        self._init_weight()

    def forward(self, x):
        x0 = self.global_pool(x)
        xs = [aspp(x) for aspp in self.aspps]
        x0 = F.interpolate(x0, size=xs[0].size()[2:], mode='bilinear', align_corners=True)
        x = torch.cat([x0] + xs, dim=1)
        return self.out_conv(x)
    
    def _init_weight(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                torch.nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()


## UneXt50

In [15]:
class UneXt50(nn.Module):
    def __init__(self, stride=1, **kwargs):
        super().__init__()
        #encoder
        # m = torch.hub.load('facebookresearch/semi-supervised-ImageNet1K-models',
        #                    'resnext50_32x4d_ssl')
        m = timm.create_model("swsl_resnext50_32x4d", pretrained=True)
        self.enc0 = nn.Sequential(m.conv1, m.bn1, nn.ReLU(inplace=True))
        self.enc1 = nn.Sequential(nn.MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1),
                            m.layer1) #256
        self.enc2 = m.layer2 #512
        self.enc3 = m.layer3 #1024
        self.enc4 = m.layer4 #2048
        #aspp with customized dilatations
        self.aspp = ASPP(2048,256,out_c=512,dilations=[stride*1,stride*2,stride*3,stride*4])
        self.drop_aspp = nn.Dropout2d(0.5)
        #decoder
        self.dec4 = UnetBlock(512,1024,256)
        self.dec3 = UnetBlock(256,512,128)
        self.dec2 = UnetBlock(128,256,64)
        self.dec1 = UnetBlock(64,64,32)
        self.fpn = FPN([512,256,128,64],[16]*4)
        self.drop = nn.Dropout2d(0.1)
        self.final_conv = ConvLayer(32+16*4, 12, ks=1, norm_type=None, act_cls=None)
        
    def forward(self, x):
        enc0 = self.enc0(x)
        enc1 = self.enc1(enc0)
        enc2 = self.enc2(enc1)
        enc3 = self.enc3(enc2)
        enc4 = self.enc4(enc3)
        enc5 = self.aspp(enc4)
        dec3 = self.dec4(self.drop_aspp(enc5),enc3)
        dec2 = self.dec3(dec3,enc2)
        dec1 = self.dec2(dec2,enc1)
        dec0 = self.dec1(dec1,enc0)
        x = self.fpn([enc5, dec3, dec2, dec1], dec0)
        x = self.final_conv(self.drop(x))
        x = F.interpolate(x,scale_factor=2,mode='bilinear')
        return x

In [16]:
class DiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1e-7):
        
        inputs = inputs.log_softmax(dim=1).exp()
        
        bs = targets.size(0)
        num_classes = inputs.size(1)
        dims = (0, 2)
        
        
        targets = targets.view(bs, -1)
        inputs = inputs.view(bs, num_classes, -1)
        
        targets = F.one_hot(targets, num_classes)
        targets = targets.permute(0, 2, 1)
        
        intersection = torch.sum(inputs * targets, dim=dims)
        cardinality = torch.sum(inputs + targets, dim=dims)
        
        dice = (2.0 * intersection + smooth) / (cardinality + smooth)
        
        loss = 1 - dice
        
        return loss

In [17]:
class CustomLoss(nn.Module):
    def __init__(self):
        super(CustomLoss, self).__init__()
        
    def forward(self, inputs, targets, smooth=1e-7):
        
#         ce_loss = nn.CrossEntropy()(inputs, targets)
        ce_loss = F.cross_entropy(inputs, targets)
        
        inputs = inputs.log_softmax(dim=1).exp()
        
        bs = targets.size(0)
        num_classes = inputs.size(1)
        dims = (0, 2)
        
        targets = targets.view(bs, -1)
        inputs = inputs.view(bs, num_classes, -1)
        
        targets = F.one_hot(targets, num_classes)
        targets = targets.permute(0, 2, 1)
        
        intersection = torch.sum(inputs * targets, dim=dims)
        cardinality = torch.sum(inputs + targets, dim=dims)
        
        dice = (2.0 * intersection + smooth) / (cardinality + smooth)
        
        loss = 1 - dice

        mask = targets.sum(dims) > 0
        loss *= mask.to(loss.dtype)


        return (loss.mean()) +  ce_loss

In [18]:
def label_smoothed_nll_loss(
    lprobs: torch.Tensor, target: torch.Tensor, epsilon: float, ignore_index=None, reduction="mean", dim=-1
) -> torch.Tensor:
    """
    Source: https://github.com/pytorch/fairseq/blob/master/fairseq/criterions/label_smoothed_cross_entropy.py
    :param lprobs: Log-probabilities of predictions (e.g after log_softmax)
    :param target:
    :param epsilon:
    :param ignore_index:
    :param reduction:
    :return:
    """
    if target.dim() == lprobs.dim() - 1:
        target = target.unsqueeze(dim)

    if ignore_index is not None:
        pad_mask = target.eq(ignore_index)
        target = target.masked_fill(pad_mask, 0)
        nll_loss = -lprobs.gather(dim=dim, index=target)
        smooth_loss = -lprobs.sum(dim=dim, keepdim=True)

        # nll_loss.masked_fill_(pad_mask, 0.0)
        # smooth_loss.masked_fill_(pad_mask, 0.0)
        nll_loss = nll_loss.masked_fill(pad_mask, 0.0)
        smooth_loss = smooth_loss.masked_fill(pad_mask, 0.0)
    else:
        nll_loss = -lprobs.gather(dim=dim, index=target)
        smooth_loss = -lprobs.sum(dim=dim, keepdim=True)

        nll_loss = nll_loss.squeeze(dim)
        smooth_loss = smooth_loss.squeeze(dim)

    if reduction == "sum":
        nll_loss = nll_loss.sum()
        smooth_loss = smooth_loss.sum()
    if reduction == "mean":
        nll_loss = nll_loss.mean()
        smooth_loss = smooth_loss.mean()

    eps_i = epsilon / lprobs.size(dim)
    loss = (1.0 - epsilon) * nll_loss + eps_i * smooth_loss
    return loss

class SoftCrossEntropyLoss(nn.Module):

    __constants__ = ["reduction", "ignore_index", "smooth_factor"]

    def __init__(
        self,
        reduction: str = "mean",
        smooth_factor: Optional[float] = None,
        ignore_index: Optional[int] = -100,
        dim: int = 1,
    ):
        """Drop-in replacement for torch.nn.CrossEntropyLoss with label_smoothing
        
        Args:
            smooth_factor: Factor to smooth target (e.g. if smooth_factor=0.1 then [1, 0, 0] -> [0.9, 0.05, 0.05])
        
        Shape
             - **y_pred** - torch.Tensor of shape (N, C, H, W)
             - **y_true** - torch.Tensor of shape (N, H, W)
        Reference
            https://github.com/BloodAxe/pytorch-toolbelt
        """
        super().__init__()
        self.smooth_factor = smooth_factor
        self.ignore_index = ignore_index
        self.reduction = reduction
        self.dim = dim

    def forward(self, y_pred: torch.Tensor, y_true: torch.Tensor) -> torch.Tensor:
        log_prob = F.log_softmax(y_pred, dim=self.dim)
        return label_smoothed_nll_loss(
            log_prob,
            y_true,
            epsilon=self.smooth_factor,
            ignore_index=self.ignore_index,
            reduction=self.reduction,
            dim=self.dim,
        )

In [19]:
criterion = None
if CFG['criterion'] == "DiceLoss":
    criterion = DiceLoss()
elif CFG['criterion'] == "CELoss":
    criterion = nn.CrossEntropyLoss()
elif CFG['criterion'] == "CustomLoss":
    criterion = CustomLoss()
elif CFG['criterion'] == "SoftCELoss":
    criterion = SoftCrossEntropyLoss(smooth_factor=0.1)

In [20]:
class CustomCosineAnnealingWarmUpRestarts(_LRScheduler):
    def __init__(self, optimizer, T_0, T_mult=1, eta_max=0.1, T_up=0, gamma=1., last_epoch=-1):
        if T_0 <= 0 or not isinstance(T_0, int):
            raise ValueError("Expected positive integer T_0, but got {}".format(T_0))
        if T_mult < 1 or not isinstance(T_mult, int):
            raise ValueError("Expected integer T_mult >= 1, but got {}".format(T_mult))
        if T_up < 0 or not isinstance(T_up, int):
            raise ValueError("Expected positive integer T_up, but got {}".format(T_up))
        self.T_0 = T_0
        self.T_mult = T_mult
        self.base_eta_max = eta_max
        self.eta_max = eta_max
        self.T_up = T_up
        self.T_i = T_0
        self.gamma = gamma
        self.cycle = 0
        self.T_cur = last_epoch
        super(CustomCosineAnnealingWarmUpRestarts, self).__init__(optimizer, last_epoch)
        
    
    def get_lr(self):
        if self.T_cur == -1:
            return self.base_lrs
        elif self.T_cur < self.T_up:
            return [(self.eta_max - base_lr)*self.T_cur / self.T_up + base_lr for base_lr in self.base_lrs]
        else:
            return [base_lr + (self.eta_max - base_lr) * (1 + math.cos(math.pi * (self.T_cur-self.T_up) / (self.T_i - self.T_up))) / 2
                    for base_lr in self.base_lrs]

    def step(self, epoch=None):
        if epoch is None:
            epoch = self.last_epoch + 1
            self.T_cur = self.T_cur + 1
            if self.T_cur >= self.T_i:
                self.cycle += 1
                self.T_cur = self.T_cur - self.T_i
                self.T_i = (self.T_i - self.T_up) * self.T_mult + self.T_up
        else:
            if epoch >= self.T_0:
                if self.T_mult == 1:
                    self.T_cur = epoch % self.T_0
                    self.cycle = epoch // self.T_0
                else:
                    n = int(math.log((epoch / self.T_0 * (self.T_mult - 1) + 1), self.T_mult))
                    self.cycle = n
                    self.T_cur = epoch - self.T_0 * (self.T_mult ** n - 1) / (self.T_mult - 1)
                    self.T_i = self.T_0 * self.T_mult ** (n)
            else:
                self.T_i = self.T_0
                self.T_cur = epoch
                
        self.eta_max = self.base_eta_max * (self.gamma**self.cycle)
        self.last_epoch = math.floor(epoch)
        for param_group, lr in zip(self.optimizer.param_groups, self.get_lr()):
            param_group['lr'] = lr

In [21]:
def _fast_hist(label_true, label_pred, n_class):
    mask = (label_true >= 0) & (label_true < n_class)
    hist = np.bincount(
        n_class * label_true[mask].astype(int) +
        label_pred[mask], minlength=n_class ** 2).reshape(n_class, n_class)
    return hist


def label_accuracy_score_(label_trues, label_preds, n_class):
    """Returns accuracy score evaluation result.
      - overall accuracy
      - mean accuracy
      - mean IU
      - fwavacc
    """
    hist = np.zeros((n_class, n_class))
    for lt, lp in zip(label_trues, label_preds):
        hist += _fast_hist(lt.flatten(), lp.flatten(), n_class)
    acc = np.diag(hist).sum() / hist.sum()
    with np.errstate(divide='ignore', invalid='ignore'):
        acc_cls = np.diag(hist) / hist.sum(axis=1)
    acc_cls = np.nanmean(acc_cls)
    with np.errstate(divide='ignore', invalid='ignore'):
        iu = np.diag(hist) / (
            hist.sum(axis=1) + hist.sum(axis=0) - np.diag(hist)
        )
    mean_iu = np.nanmean(iu)
    freq = hist.sum(axis=1) / hist.sum()
    fwavacc = (freq[freq > 0] * iu[freq > 0]).sum()
    return acc, acc_cls, mean_iu, fwavacc

In [22]:
def label_accuracy_score(hist):
    """
    Returns accuracy score evaluation result.
      - [acc]: overall accuracy
      - [acc_cls]: mean accuracy
      - [mean_iu]: mean IU
      - [fwavacc]: fwavacc
    """
    acc = np.diag(hist).sum() / hist.sum()
    with np.errstate(divide='ignore', invalid='ignore'):
        acc_cls = np.diag(hist) / hist.sum(axis=1)
    acc_cls = np.nanmean(acc_cls)

    with np.errstate(divide='ignore', invalid='ignore'):
        iu = np.diag(hist) / (hist.sum(axis=1) + hist.sum(axis=0) - np.diag(hist))
    mean_iu = np.nanmean(iu)

    freq = hist.sum(axis=1) / hist.sum()
    fwavacc = (freq[freq > 0] * iu[freq > 0]).sum()
    return acc, acc_cls, mean_iu, fwavacc

In [23]:
def add_hist(hist, label_trues, label_preds, n_class):
    """
        stack hist(confusion matrix)
    """

    for lt, lp in zip(label_trues, label_preds):
        hist += _fast_hist(lt.flatten(), lp.flatten(), n_class)

    return hist

In [24]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1.0 - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)
    return bbx1, bby1, bbx2, bby2

In [25]:
def cutmix(data, target, alpha):
    indices = torch.randperm(data.size(0))
    shuffled_data = data[indices]
    shuffled_target = target[indices]

    lam = np.clip(np.random.beta(alpha, alpha), 0.3, 0.4)
    bbx1, bby1, bbx2, bby2 = rand_bbox(data.size(), lam)
    new_data = data.clone()
    new_target = target.clone()
    new_data[:, :, bby1:bby2, bbx1:bbx2] = data[indices, :, bby1:bby2, bbx1:bbx2]
    new_target[:, bby1:bby2, bbx1:bbx2] = target[indices, bby1:bby2, bbx1:bbx2]
    # adjust lambda to exactly match pixel ratio
    lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (data.size()[-1] * data.size()[-2]))
    

    return new_data, new_target

In [26]:
def train_one_epoch(epoch, model, device, optimizer, criterion, train_loader, scheduler):
    model.train()
    running_loss = None
    
    pbar = tqdm(enumerate(train_loader), total=len(train_loader), position=0, leave=True)

    for step, (imgs, masks) in pbar:

        if (step+1) == (len(train_loader)):
            continue
        mix_decision = np.random.rand()
        imgs = torch.stack(imgs)
        masks = torch.stack(masks)
        imgs = imgs.to(device).float()
        masks = masks.to(device).long()
        
        if mix_decision < CFG['mix_prob']:
            imgs, masks = cutmix(imgs, masks, 1.0)

        with autocast():
            model.to(device)
            mask_preds = model(imgs)
            loss = criterion(mask_preds, masks) / CFG['gradient_accumulation_steps']
            scaler.scale(loss).backward()

            # loss.backward()

            if running_loss is None:
                running_loss = loss.item() * CFG['gradient_accumulation_steps']
            else:
                running_loss = running_loss * 0.99 + loss.item() * CFG['gradient_accumulation_steps'] * 0.01

            if ((step + 1) % CFG["gradient_accumulation_steps"]==0) or ((step+1) == (len(train_loader))):
                scaler.step(optimizer)
                scaler.update()
                # optimizer.zero_grad()
                # optimizer.step()                            
                optimizer.zero_grad() 
                description = f"epoch {epoch} loss: {running_loss: .4f}"
                pbar.set_description(description)
                
    scheduler.step()        

In [27]:
def valid_one_epoch(epoch, model, device, criterion, val_loader):
    model.eval()
    
    total_loss = 0
    running_loss = None
    cnt = 0
    mIoU_list = []
    pbar = tqdm(enumerate(val_loader), total=len(val_loader), position=0, leave=True)
    hist = np.zeros((12, 12))
    for step, (imgs, masks) in pbar:
        if (step+1) == (len(train_loader)):
            continue
        imgs = torch.stack(imgs)
        masks = torch.stack(masks)
        imgs = imgs.to(device).float()
        masks = masks.to(device).long()
        
        cnt += 1

        mask_preds = model(imgs)
        # print(f"{mask_preds.shape}       ")
        loss = criterion(mask_preds, masks)

        mask_preds = torch.argmax(mask_preds, dim=1).detach().cpu().numpy()
        # print(mask_preds.shape)

        mIoU = label_accuracy_score_(masks.detach().cpu().numpy(), mask_preds, n_class=12)[2]
        mIoU_list.append(mIoU)

        total_loss += loss.item()
            
        if running_loss is None:
            running_loss = loss.item()
        else:
            running_loss = running_loss * 0.99 + loss.item() * 0.01

        description = f'epoch {epoch} Loss: {running_loss:.4f}, mIoU: {np.mean(mIoU_list):.4f}'
        pbar.set_description(description)

    return total_loss/cnt, np.mean(mIoU_list)

In [28]:
FOLDS = 5
kf = KFold(FOLDS, shuffle=True, random_state=CFG['seed'])
df["Folds"] = 0

for fold, (train_idx, val_idx) in enumerate(kf.split(df)):
    df.loc[val_idx, 'Folds'] = fold

In [29]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [30]:
def create_folder(directory):
    try:
        os.makedirs(directory)
    except:
        pass

In [ ]:
for fold in range(FOLDS):
    # if fold == 0 or fold == 1 or fold == 2:
    #   continue
    print(f"{fold} fold start")
    
    if CFG['decoder'] == "Unetpp":
        model = smp.UnetPlusPlus(CFG['encoder'], encoder_weights=CFG['pretrained'], in_channels=3, classes=12).to(device)
    elif CFG['decoder'] == 'DeepLabV3Plus':
        model = smp.DeepLabV3Plus(CFG['encoder'], encoder_weights=CFG['pretrained'], in_channels=3, classes=12).to(device)
    elif CFG['decoder'] == 'DeepLabV3':
        model = smp.DeepLabV3(CFG['encoder'], encoder_weights=CFG['pretrained'], in_channels=3, classes=12).to(device)
    elif CFG['decoder'] == "UperNet":
        model = encoding.models.sseg.UperNet(12, CFG['encoder'], aux=False)
    elif CFG['decoder'] == "PSPNet":
        model = smp.PSPNet(CFG['encoder'], encoder_weights=CFG['pretrained'], in_channels=3, classes=12).to(device)
    elif CFG['decoder'] == "UneXt":
        model = UneXt50()
    
    if CFG['scheduler'] == "Warmup":
        if CFG['optimizer'] == "Adam":
            optimizer = torch.optim.Adam(model.parameters(), lr=0, weight_decay=CFG['weight_decay'])
            scheduler = CustomCosineAnnealingWarmUpRestarts(optimizer, T_0=CFG['epochs'], T_mult=1, eta_max=CFG['lr'], T_up=CFG['epochs']//5, gamma=1.)
        elif CFG['optimizer'] == "AdamW":
            optimizer = torch.optim.AdamW(model.parameters(), lr=0, weight_decay=CFG['weight_decay'])
            scheduler = CustomCosineAnnealingWarmUpRestarts(optimizer, T_0=CFG['epochs'], T_mult=1, eta_max=CFG['lr'], T_up=CFG['epochs']//5, gamma=1.)
    else:
        if CFG['optimizer'] == "Adam":
            optimizer = torch.optim.Adam(model.parameters(), lr=CFG['lr'], weight_decay=CFG['weight_decay'])
    
    scaler = GradScaler()
    train_loader, valid_loader = prepare_dataloader(df, [fold])
    
    best_mIoU = 0
    num_epochs = CFG['epochs']
    
    for epoch in range(num_epochs):
        train_one_epoch(epoch, model, device, optimizer, criterion, train_loader, scheduler)

        with torch.no_grad():
            epoch_loss, mIoU = valid_one_epoch(epoch, model, device, criterion, valid_loader)

        # neptune.log_metric(f"fold{fold} epoch loss", epoch_loss)
        # neptune.log_metric(f"fold{fold} mIoU", mIoU)

        if best_mIoU < mIoU:
            best_mIoU = mIoU
            dir_ = f"/content/drive/MyDrive/trash_segmentation/models"
            create_folder(dir_)
            # torch.save({'model': model.state_dict(),
            #             'optimizer': optimizer.state_dict(),
            #             'scheduler': scheduler.state_dict()
            #             }, f"{dir_}/{CFG['decoder']}_{CFG['encoder']}_{fold}.pth")
            torch.save(model.state_dict(), f"{dir_}/{CFG['decoder']}_{CFG['encoder']}_{fold}.pth")
            print("model is saved")
        print("")

    # neptune.log_metric(f"fold {fold} Best mIoU", best_mIoU)
    del model, optimizer, train_loader, valid_loader, scheduler
    gc.collect()
    torch.cuda.empty_cache()
# neptune.stop()

0 fold start


  0%|          | 0/411 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
epoch 0 Loss: 2.7002, mIoU: 0.0128: 100%|██████████| 103/103 [00:18<00:00,  5.44it/s]


model is saved



epoch 1 Loss: 0.8700, mIoU: 0.2394: 100%|██████████| 103/103 [00:19<00:00,  5.40it/s]


model is saved



...

epoch 8 Loss: 0.7591, mIoU: 0.4761: 100%|██████████| 103/103 [00:18<00:00,  5.50it/s]


epoch 9 Loss: 0.7470, mIoU: 0.5191: 100%|██████████| 103/103 [00:18<00:00,  5.48it/s]


model is saved



epoch 10 Loss: 0.7649, mIoU: 0.5188: 100%|██████████| 103/103 [00:19<00:00,  5.36it/s]


epoch 11 loss:  0.6488:  99%|█████████▊| 405/411 [02:24<00:02,  2.83it/s]